In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
sc = SparkContext('local[8]', 'Assignment_2')

In [ ]:
import logging
import traceback
import json
logger = logging.getLogger('py4j')
import itertools

In [ ]:
sample_data = sc.textFile('asnlib/publicdata/small1.csv').map(lambda x: x.split(',')).map(lambda x: (x[0], x[1]))
sample_data.first()

In [ ]:
sample_data.take(10)

In [ ]:
sample_data.count()

In [ ]:
header = sample_data.first()
sample_data_without_header = sample_data.filter(lambda x: x!= header)

In [ ]:
sample_data_without_header.first()

In [ ]:
distinct_sets = sample_data_without_header.distinct()

In [ ]:
distinct_sets.count()

In [ ]:
grouped_sets_2 = distinct_sets.map(lambda x: (x[1],x[0])).groupByKey().map(lambda x: (x[0], list(x[1])))

In [ ]:
grouped_sets_2.collect()

In [ ]:
grouped_sets = distinct_sets.groupByKey().map(lambda x: (x[0], sorted(list(x[1]))))

In [ ]:
grouped_sets.collect()

In [ ]:
grouped_sets.glom().map(len).collect()

In [ ]:
support = 4
n = grouped_sets.getNumPartitions()
part_support = support//n 


In [ ]:
def get_counts(basket):
    counts = []
    print(basket)
    for item in basket[1]:
        counts.append((item,1))
        print(item)
    return counts

In [ ]:
sets = grouped_sets.flatMap(get_counts)

In [ ]:
sets.take(10)

In [ ]:
summed_occurences = sets.reduceByKey(lambda a,b : a+b)

In [ ]:
singletons = summed_occurences.filter(lambda x: x[1] >= support)

In [ ]:
def get_partition_candidates(bucket):
    items = {}
    for basket in bucket:
        for item in basket[1]:
            if item not in items:
                items[item] = 0
            items[item] += 1
    print(items)
    for item in items:
        if items[item] >= part_support:
            yield item

In [ ]:
candidate = grouped_sets.mapPartitions(get_partition_candidates)

In [ ]:
candidate

In [ ]:
candidates = candidate.distinct().collect()

In [ ]:
part_support

In [ ]:
candidates

In [ ]:
original_freq_sets = sample_data_without_header.filter(lambda x: x[1] in candidates).distinct().groupByKey().map(lambda x: (x[0], list(x[1]))).flatMap(get_counts).reduceByKey(lambda a,b: a+b)

In [ ]:
original_freq_sets.collect()

In [ ]:
candidates

In [ ]:
original_freq_sets.filter(lambda x: x[1] >= support).map(lambda x: (x[0])).collect()

In [ ]:
def get_partition_itemset_candidates(bucket, size_of_set):
    items = {}
    for basket in bucket:
        if len(basket[1]) < size_of_set:
            continue
        sets = itertools.combinations(basket[1], size_of_set)
        for item in sets:
#             item = sorted(item)
            if item not in items:
                items[item] = 0
            items[item] += 1
            
    print(items)
    for item in items:
        if items[item] >= part_support:
            yield item

def get_original_itemset_counts(basket, size_of_set):
    sets = itertools.combinations(basket[1], size_of_set)
    return list(map(lambda x: (x,1), sets))
#     counts = []
#     print(basket)
#     for item in basket[1]:
#         counts.append((item,1))
#         print(item)
#     return counts

In [ ]:
candidate_pairs = grouped_sets.mapPartitions(lambda x: get_partition_itemset_candidates(x,2))

In [ ]:
candidate_pairs_list = candidate_pairs.distinct().collect()

In [ ]:
candidate_pairs_list

In [ ]:
original_freq_pairs = grouped_sets.flatMap(lambda x: get_original_itemset_counts(x,2)).filter(lambda x: x[0] in candidate_pairs_list).reduceByKey(lambda a,b: a+b)

In [ ]:
original_freq_pairs.filter(lambda x: x[1] >= support).collect()

In [ ]:
candidate_triplets = grouped_sets.mapPartitions(lambda x: get_partition_itemset_candidates(x,3))

In [ ]:
candidate_triplets_list = candidate_triplets.distinct().collect()

In [ ]:
candidate_triplets_list

In [ ]:
len(candidate_triplets_list)

In [ ]:
len(candidate_pairs_list)

In [ ]:
original_freq_triplets = grouped_sets.flatMap(lambda x: get_original_itemset_counts(x,3)).filter(lambda x: x[0] in candidate_triplets_list).reduceByKey(lambda a,b: a+b)

In [ ]:
original_freq_triplets.filter(lambda x: x[1] >= support).collect()

In [ ]:
candidate_sets = grouped_sets.mapPartitions(lambda x: get_partition_itemset_candidates(x,4))
candidate_list = candidate_sets.distinct().collect()
original_freq_sets = grouped_sets.flatMap(lambda x: get_original_itemset_counts(x,4)).filter(lambda x: x[0] in candidate_list).reduceByKey(lambda a,b: a+b)
original_freq_sets.filter(lambda x: x[1] >= support).collect()

In [ ]:
original_freq_sets.collect()

In [ ]:
candidate_list